sparql-client 설치

In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Asturias> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["label"]["value"])

Asturias
منطقة أشتورية
Astúries
Asturie
Asturien
Αστούριες
Asturio
Asturias
Asturiasko Printzerria
Asturies
Asturias
Asturias
アストゥリアス州
Asturie
아스투리아스 지방
Asturië (regio)
Asturia
Astúrias
Астурия
Asturien
Астурія
阿斯图里亚斯


In [24]:
class VisitKorea:
    def __init__(self):
        self.init_url="http://data.visitkorea.or.kr/sparql"
        self.sq=SPARQLWrapper(self.init_url)
        self.prefix="""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX vi: <http://www.saltlux.com/transformer/views#>
PREFIX kto: <http://data.visitkorea.or.kr/ontology/>
PREFIX ktop: <http://data.visitkorea.or.kr/property/>
PREFIX ids: <http://data.visitkorea.or.kr/resource/>
PREFIX wgs: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.saltlux.com/geo/property#>
PREFIX pf: <http://www.saltlux.com/DARQ/property#> 
"""
    def query(self,q):
        q=self.prefix+q
        sparql=self.sq
        sparql.setQuery(q)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        return results["results"]["bindings"]

In [49]:
q1="""
SELECT * 
WHERE {
 ?resource a kto:Place ;
      wgs:lat ?lat1 ;
      wgs:long ?long ;
      rdfs:label ?name .
 FILTER contains(str(?name),'일민미술관')
 
} limit 10
"""

In [50]:
client=VisitKorea()

In [51]:
results=client.query(q1)

In [52]:
results

[{'resource': {'type': 'uri',
   'value': 'http://data.visitkorea.or.kr/resource/130227'},
  'lat1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'typed-literal',
   'value': '37.5698760651'},
  'long': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'typed-literal',
   'value': '126.9776487818'},
  'name': {'type': 'literal', 'xml:lang': 'ko', 'value': '일민미술관'}}]

In [28]:
for r in results:
    print(r['resource']['value'],r['name']['value'])

http://data.visitkorea.or.kr/resource/129490 응봉산(울진)
http://data.visitkorea.or.kr/resource/130771 서울특별시교육청 구로도서관
http://data.visitkorea.or.kr/resource/130227 일민미술관
http://data.visitkorea.or.kr/resource/130448 서대문자연사박물관
http://data.visitkorea.or.kr/resource/125596 백덕산(평창)
http://data.visitkorea.or.kr/resource/130494 이중섭 미술관
http://data.visitkorea.or.kr/resource/130416 사비나미술관
http://data.visitkorea.or.kr/resource/1912356 양산시립박물관
http://data.visitkorea.or.kr/resource/126294 야미도
http://data.visitkorea.or.kr/resource/126041 황매산(산청)


<http://data.visitkorea.or.kr/resource/A05020100>는 식당유형이 한식입니다.


In [88]:
q2="""
SELECT *
WHERE {
    ?resource a kto:Place;
              rdfs:label ?name .
    FILTER regex(?name,'^%s') .
} limit 2
"""

In [67]:
import re

In [72]:
def get_id(x):
    p=re.compile('[0-9]+$')
    o=p.findall(x)
    if len(o)>0:
        return o[0]
    pass

In [89]:
results=client.query(q2%("경복궁",))
for r in results:
    id=get_id(r['resource']['value'])
    n=r['name']['value']
    print(id,'\t',n)

729811 	 경복궁
126508 	 경복궁


In [96]:
q3="""
SELECT ?name ?near_name ?distance 
WHERE {
 ids:%d wgs:lat ?lat1 ;
           wgs:long ?long1 ;
           rdfs:label ?name . 
 (?near ?distance) geo:nearby(?lat1 ?long1 "1600") .
 ?near rdfs:label ?near_name ;
       ktop:category ids:A05020100 . 
} limit 5
"""

In [97]:
results=client.query(q3%(729811,))
for r in results:
    name=r['name']['value']
    near_name=r['near_name']['value']
    distance=r['distance']['value']
    print("%s\t%s\t%s"%(name,near_name,distance))

경복궁	장터	511.29956
경복궁	담양숯불갈비	872.8764
경복궁	경복궁	0.0
경복궁	황금어장	314.0314
경복궁	사계절복집	430.1987
